In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
import scanpy as sc 
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys 
sys.path.append('../../src')

In [5]:
import sys
sys.path.append('../../src/')
# import celloracle as co

In [6]:
adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/snrna_human_tonsil.h5ad')
adata

AnnData object with n_obs × n_vars = 5778 × 3549
    obs: 'cell_type', 'author_cell_type', 'cell_type_int', 'leiden', 'leiden_R', 'cell_type_2'
    uns: 'author_cell_type_colors', 'cell_type_2_colors', 'cell_type_colors', 'dendrogram_leiden', 'leiden', 'leiden_R', 'leiden_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'ora_estimate', 'ora_pvals', 'spatial', 'spatial_unscaled'
    varm: 'PCs'
    layers: 'imputed_count', 'normalized_count'
    obsp: 'connectivities', 'distances'

In [7]:
from spaceoracle.tools.network import RegulatoryFactory
co_grn = RegulatoryFactory(
    colinks_path='/ix/djishnu/shared/djishnu_kor11/training_data_2025/snrna_human_tonsil_colinks.pkl',
    annot='cell_type_int'
)

In [8]:
from spaceoracle.models.parallel_estimators import *

# cell_threshes = pd.read_csv('../../data/tonsil/cell_threshes.csv', index_col=0)
# adata.obs['cell_thresholds'] = cell_threshes.loc[adata.obs_names, '0']
# cell_threshes.mean().item()

cell_threshes = pd.read_parquet('/ix/djishnu/shared/djishnu_kor11/commot_outputs/tonsil_LRs.parquet')
adata.uns['cell_thresholds'] = cell_threshes
adata.uns['cell_thresholds'].shape

(5778, 958)

In [9]:
from spaceoracle.models.parallel_estimators import SpatialCellularProgramsEstimator

estimator = SpatialCellularProgramsEstimator(
    adata, target_gene='PAX5', cluster_annot='cell_type_int',
    radius=200, contact_distance=30, grn=co_grn)

In [18]:
estimator.fit(num_epochs=100, learning_rate=5e-3, score_threshold=0.1, coef_filter=0.001)
estimator.betadata.to_parquet('filtered_betadata.parquet')

Fitting PAX5 with 278 modulators
	22 Transcription Factors
	244 Ligand-Receptor Pairs
	12 TranscriptionFactor-Ligand Pairs
0: 0.8232 | 0.8068
1: 0.9241 | 0.9049
2: 0.7577 | 0.6993
3: 0.9656 | 0.9449
4: 0.9085 | 0.9069
5: 0.9977 | 0.9982
6: 0.9933 | 0.9939
7: 0.9508 | 0.9207
8: 0.9214 | 0.8860


In [10]:
# df = received_ligands(
#     adata.obsm['spatial'], 
#     get_ligands_df(adata.to_df(layer='imputed_count'), adata.obs['cell_thresholds'], estimator.ligands),
#     lr_info=estimator.lr 
# )

# df_nofilt = received_ligands(
#     adata.obsm['spatial'], 
#     get_ligands_df(adata.to_df(layer='imputed_count'), np.zeros(adata.n_obs), estimator.ligands),
#     lr_info=estimator.lr 
# )

# (df - df_nofilt).sum(axis=0)

In [15]:
adata.uns['cell_thresholds'] = None

estimator.fit(num_epochs=100, learning_rate=5e-3, score_threshold=0.1, coef_filter=0.001)
estimator.betadata.to_parquet('unfiltered_betadata.parquet')

Fitting PAX5 with 278 modulators
	22 Transcription Factors
	244 Ligand-Receptor Pairs
	12 TranscriptionFactor-Ligand Pairs
0: 0.8716 | 0.8645
1: 0.9456 | 0.9351
2: 0.9449 | 0.9220
3: 0.9702 | 0.9470
4: 0.9396 | 0.9425
5: 0.9987 | 0.9983
6: 0.9944 | 0.9955
7: 0.9548 | 0.8869
8: 0.9544 | 0.9246


In [10]:
from spaceoracle.models.parallel_estimators import *

df = get_filtered_df(
    adata.to_df(layer='imputed_count'), 
    cell_threshes, 
    genes=adata.var_names
)

df.columns[df.sum(axis=0) == 0]

Index(['A1BG', 'A2M', 'A2ML1', 'ADAM12', 'ADAM21', 'ADAM22', 'ADAM23',
       'ADAM29', 'ADAM8', 'ADAMTS10',
       ...
       'ZSCAN16', 'ZSCAN18', 'ZSCAN2', 'ZSCAN25', 'ZSCAN29', 'ZSCAN32',
       'ZSWIM5', 'ZSWIM6', 'ZSWIM7', 'ZXDB'],
      dtype='object', name='Unnamed: 0', length=3351)

In [11]:
tmp = df[['CD40', 'CD40LG']]
tmp['cell_type'] = adata.obs['cell_type']
tmp.groupby('cell_type').sum()

Unnamed: 0,CD40,CD40LG
cell_type,,
B memory,383.452242,0.000000
B naive,338.164137,0.000000
CD4+ T,0.000000,115.127196
DC,169.188907,28.212610
GC B,1311.768740,0.000000
Macrophages,29.006237,1.170220
Plasma,247.238076,28.929232
T cells,0.000000,141.203411
Tfh,0.000000,199.324654


In [9]:
from spaceoracle import SpaceTravLR

cell_threshes = pd.read_parquet('/ix/djishnu/shared/djishnu_kor11/commot_outputs/tonsil_LRs.parquet')
adata.uns['cell_thresholds'] = cell_threshes

star = SpaceTravLR(
    adata=adata,
    annot='cell_type_int', 
    max_epochs=100, 
    learning_rate=5e-4, 
    spatial_dim=64,
    batch_size=512,
    grn=co_grn,
    radius=150,
    contact_distance=30,
    save_dir='/ix/djishnu/shared/djishnu_kor11/super_filtered_runs/human_tonsil_v5'
)

star.run()

Fitting KLHL29 with 326 modulators
	29 Transcription Factors
	244 Ligand-Receptor Pairs
	53 TranscriptionFactor-Ligand Pairs
0: 0.6947 | 0.7164
1: 0.8051 | 0.8106
2: 0.6922 | 0.7220
3: 0.9957 | 0.9975
4: 0.9518 | 0.9519
5: 0.9957 | 0.9971
6: 0.9777 | 0.9854
7: 0.9277 | 0.9305
8: 0.4202 | 0.7210
Fitting PTPRT with 320 modulators
	27 Transcription Factors
	244 Ligand-Receptor Pairs
	49 TranscriptionFactor-Ligand Pairs
0: 0.1970 | 0.5470
1: 0.6046 | 0.7036
2: x.xxxx | 0.4141
3: 0.9987 | 0.9997
4: 0.7009 | 0.7510
5: 0.9366 | 0.9727
6: 0.9944 | 0.9979
7: 0.3128 | 0.6256
8: 0.1795 | 0.7771
Fitting GSG1L with 432 modulators
	54 Transcription Factors
	244 Ligand-Receptor Pairs
	134 TranscriptionFactor-Ligand Pairs
0: x.xxxx | 0.3629
1: x.xxxx | 0.1969
2: x.xxxx | 0.2707
3: x.xxxx | 0.9713
4: 0.3028 | 0.6105
5: x.xxxx | 0.9986
6: x.xxxx | 0.8800
7: x.xxxx | 0.5299
8: x.xxxx | 0.5906
Fitting FMN1 with 346 modulators
	34 Transcription Factors
	244 Ligand-Receptor Pairs
	68 TranscriptionFactor-Lig

In [ ]:
_receptors = np.unique(estimator.lr.receptor.values)
_layer = 'normalized_count' if 'normalized_count' in estimator.adata.layers else 'imputed_count'
receptor_levels = estimator.adata.to_df(layer=_layer)[np.unique(_receptors)].join(
    estimator.adata.obs[estimator.cluster_annot]).groupby(estimator.cluster_annot).mean().max(0).to_frame()
receptor_levels.columns = ['mean_max']

In [ ]:
receptor_levels[receptor_levels.mean_max < 0.2]

In [ ]:
# %matplotlib inline
# estimator.plot_modulators()

In [ ]:
estimator.fit(num_epochs=100, learning_rate=5e-3, score_threshold=0.1, coef_filter=0.001)

In [ ]:
betadata = estimator.betadata

In [ ]:
anchor = estimator.models[0].anchors.cpu().numpy()

In [ ]:
estimator.adata.obs['cell_type_int'].value_counts()

In [ ]:
pd.DataFrame(np.where(np.abs(anchor) < (1/100.0), 0, anchor)[1:], 
    index=estimator.modulators).abs().sort_values(by=0, ascending=True)

In [ ]:
beta_cols = [i for i in betadata.columns if 'beta_' in i]

In [ ]:
betadata[beta_cols].mean().sort_values(ascending=False)

In [ ]:
from spaceoracle.prophets import Prophet
import anndata as ad
import pandas as pd
import matplotlib

In [ ]:
betadata.to_parquet(f'/tmp/models/{estimator.target_gene}_betadata.parquet')

In [ ]:
pythia = Prophet(
    adata=estimator.adata,
    models_dir='/tmp/models',
    annot='cell_type_int',
    annot_labels='cell_type'
)

In [ ]:
gex_df = estimator.adata.to_df(layer=estimator.layer)
pythia.compute_betas()
gene_mtx = pythia.adata.layers['imputed_count']
weighted_ligands = pythia._compute_weighted_ligands(gene_mtx)
beta_dict = pythia._get_wbetas_dict(
    pythia.beta_dict, weighted_ligands, gene_mtx)
wbetas = beta_dict.data[estimator.target_gene].wbetas
xy = beta_dict.xydf.copy()
xy.columns = ['x', 'y']
df = wbetas \
        .join(estimator.adata.obs) \
        .join(xy) \
        .join(gex_df)

In [ ]:
beta_cols_df = [i for i in df.columns if 'beta_' in i and i.replace('beta_', '') in np.unique(estimator.ligands)]

In [ ]:
beta_cols = [i for i in betadata.columns if 'beta_' in i and '$' in i]

In [ ]:
betadata[beta_cols].mean().sort_values(ascending=False)

In [ ]:
df[['beta_GAS6', 'beta_NPPC']].mean()

In [ ]:
# Create figure with 4 subplots in 2x2 layout
fig, axes = plt.subplots(2, 2, figsize=(14, 10), dpi=200)
axes = axes.flatten()

modulator = 'CCL21'

# Join data once for efficiency
plot_data = df[[f'beta_{modulator}', 'x', 'y', 'cell_type']].join(adata.to_df(layer='imputed_count'))

# Common plot settings
plot_settings = {
    'linewidth': 0.2,
    'edgecolor': 'black',
    's': 30
}

# Plot for cell_type
sns.scatterplot(
    data=plot_data,
    x='x',
    y='y', 
    hue='cell_type',
    palette='tab20',
    legend='brief',
    ax=axes[0],
    **plot_settings
)
axes[0].set_title('Cell Type', fontsize=14)
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot for modulator gene with colorbar
scatter1 = axes[1].scatter(
    plot_data['x'],
    plot_data['y'],
    c=plot_data[modulator],
    cmap='viridis',
    **plot_settings
)
axes[1].set_title(modulator, fontsize=14)
plt.colorbar(scatter1, ax=axes[1], shrink=0.5)

# Plot for target gene with colorbar  
scatter2 = axes[2].scatter(
    plot_data['x'],
    plot_data['y'],
    c=plot_data[estimator.target_gene], 
    cmap='magma',
    **plot_settings
)
axes[2].set_title(estimator.target_gene, fontsize=14)
plt.colorbar(scatter2, ax=axes[2], shrink=0.5)

# Plot for beta_modulator with colorbar
scatter3 = axes[3].scatter(
    plot_data['x'],
    plot_data['y'],
    c=plot_data[f'beta_{modulator}'],
    cmap='rainbow', 
    **plot_settings
)
axes[3].set_title(f'Beta {modulator}', fontsize=14)
plt.colorbar(scatter3, ax=axes[3], shrink=0.5)

# Remove ticks and set equal aspect for all axes
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_frame_on(False)

# Remove borders from legends
for ax in axes:
    legend = ax.get_legend()
    if legend is not None:
        legend.set_frame_on(False)

plt.tight_layout()
plt.show()

In [ ]:
import commot as ct

In [ ]:
df_ligrec = ct.pp.ligand_receptor_database(
        database='CellChat', 
        species='human', 
        signaling_type="Secreted Signaling"
    )
df_ligrec.columns = ['ligand', 'receptor', 'pathway', 'signaling']
          

In [ ]:
df_ligrec.query('ligand == "CCL21"')